In [1]:
import os
import numpy as np 
import torch
import torch.utils.data as Data
import torch.optim as optim

from sklearn.metrics import r2_score, accuracy_score

from MyNet import Net, MetaDataSet, MyLoss, Net_3hiddens

# hyper parameters
BATCH_SIZE = 128
EPOCH_SIZE = 1000

LR = 1e-5
MOMENTUM = 0.9
WEIGHT_DECAY =  0.0005

DATASET_SHUFFLE = False
num_WORKERS = 16

TASK = 'regression'

LAMBDA = 1
XI = 0.0005


save_path = './'
metadata_dir = '../new_bigmetadata/australian/query_time/'

best_acc = 0  # best test accuracy
best_r2 = 0 
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# net = Net(n_feature=396, first_n_hidden=100, second_n_hidden=20, n_output=2)
# net_bn = Net(n_feature=418, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=True)
# net_no_bn = Net(n_feature=418, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=False)
net = Net_3hiddens(n_feature=418, first_n_hidden=200, second_n_hidden=100, third_n_hidden=50, n_output=1, batch_normalization=True)

optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
# optimizer = optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = MyLoss()
print(net)
print(optimizer)
print(criterion)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('The current device is : ', device)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)

print('Preparing the dataset....')
# trainloader = MetaDataSet(metadata_dir, BATCH_SIZE, num_WORKERS, DATASET_SHUFFLE, TASK)
testloader = MetaDataSet(metadata_dir, BATCH_SIZE, num_WORKERS, DATASET_SHUFFLE, TASK)

metadata1 = np.load('../new_bigmetadata/australian/query_time/2australian30_big_metadata30.npy')
metadata2 = np.load('../new_bigmetadata/australian/query_time/4australian30_big_metadata30.npy')
metadata3 = np.load('../new_bigmetadata/australian/query_time/10australian30_big_metadata30.npy')
metadata4 = np.load('../new_bigmetadata/australian/query_time/20australian30_big_metadata30.npy')
metadata5 = np.load('../new_bigmetadata/australian/query_time/30australian30_big_metadata30.npy')
metadata6 = np.load('../new_bigmetadata/australian/query_time/48australian30_big_metadata30.npy')

metadata = np.vstack((metadata1, metadata2, metadata3, metadata4, metadata5, metadata6))

X = metadata[:, 0:418]
y = metadata[:, 418]

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)
X_tensor = X_tensor.float()
y_tensor = y_tensor.float()

troch_dataset = Data.TensorDataset(X_tensor, y_tensor)
testloader = Data.DataLoader(dataset=troch_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=DATASET_SHUFFLE,
                        num_workers=num_WORKERS)

print("Dataset is ready!")

Net_3hiddens(
  (input_bn): BatchNorm1d(418, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_hidden): Linear(in_features=418, out_features=200, bias=True)
  (first_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (second_hidden): Linear(in_features=200, out_features=100, bias=True)
  (second_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (third_hidden): Linear(in_features=100, out_features=50, bias=True)
  (third_bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=50, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 1e-05
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
MyLoss()
The current device is :  cpu
Preparing the dataset....


TypeError: 'NoneType' object is not subscriptable

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % (epoch+1))
    net.train()
    # 
    if epoch < EPOCH_SIZE * 0.3:
        lam = 0
        xi = 0.0005
    elif epoch < EPOCH_SIZE * 0.6:
        lam = 0.5
        xi = 0.0005
    else:
        lam = 1
        xi = 0.0001       
    running_loss = 0.0
    for step, (batch_x, batch_y) in enumerate(trainloader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
#         min_b = int(BATCH_SIZE/2)
        min_b = int(batch_y.size()[0]/2)

        pari_y = batch_y[0:min_b] - batch_y[min_b:]
        optimizer.zero_grad()
        output1 = net(batch_x[0:min_b,:])
        output2 = net(batch_x[min_b:,:])
        loss = criterion(output1, output2, pari_y, batch_y, xi, lam)
        loss.backward()
        optimizer.step()   
  
        # print statistics
        running_loss += loss.item()
        # print every 1000 mini-batches
        if step % 1000 == 999:    
            print('[%d, %5d] ,progress %.4f, mean loss: %.10f' %
                (epoch + 1, step + 1, step/len(trainloader), running_loss / 1000))
            running_loss = 0.0

def test(epoch):
    global best_acc
    net.eval()
    r2scorelist = []
    acclist = []
    with torch.no_grad():
        for step, (batch_x, batch_y) in enumerate(testloader):
            batch_x = batch_x.to(device)
    #         min_b = int(BATCH_SIZE/2)
            min_b = int(batch_y.size()[0]/2)
            pair_y = batch_y[0:min_b] - batch_y[min_b:]
            py = pair_y.detach().numpy()
            py[np.where(py>0)[0]] = 1
            py[np.where(py<=0)[0]] = 1
            output = net(batch_x)
            pred = output.cpu().numpy()
            y = batch_y.detach().numpy()
            r2score = r2_score(pred, y)
            pair_pred = pred[0:min_b] - pred[min_b:]
            pair_pred[np.where(pair_pred>0)[0]] = 1
            pair_pred[np.where(pair_pred<=0)[0]] = 1
            acc = accuracy_score(pair_pred, py)
            r2scorelist.append(r2score)
            acclist.append(acc)

    mean_acc = np.mean(acclist)
    mean_r2 = np.mean(r2scorelist)
    # Save checkpoint.
    if mean_acc > best_acc or mean_r2 > best_r2:
        print('Saving..')
        print('Current mean_rank_acc: %.10f'%mean_acc)
        print('Current mean r2 score: %.10f'%mean_r2)
        state = {
            'net': net.state_dict(),
            'mean_rank_acc': mean_acc,
            'mean_r2_score':mean_r2,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = mean_acc
        
for epoch in range(EPOCH_SIZE):
    train(epoch)
    test(epoch)


Epoch: 1
[1,  1000] ,progress 0.1973, mean loss: 0.0256997239
[1,  2000] ,progress 0.3948, mean loss: 0.0229400319
[1,  3000] ,progress 0.5923, mean loss: 0.0217247044
[1,  4000] ,progress 0.7898, mean loss: 0.0189203932
[1,  5000] ,progress 0.9874, mean loss: 0.0167302063
Saving..
Current mean_rank_acc: 1.0000000000
Current mean r2 score: -3.1047238154

Epoch: 2
[2,  1000] ,progress 0.1973, mean loss: 0.0157733812
[2,  2000] ,progress 0.3948, mean loss: 0.0148333268
[2,  3000] ,progress 0.5923, mean loss: 0.0146855708
[2,  4000] ,progress 0.7898, mean loss: 0.0131000582
[2,  5000] ,progress 0.9874, mean loss: 0.0117433289

Epoch: 3
[3,  1000] ,progress 0.1973, mean loss: 0.0114071417
[3,  2000] ,progress 0.3948, mean loss: 0.0109811597
[3,  3000] ,progress 0.5923, mean loss: 0.0111425339
[3,  4000] ,progress 0.7898, mean loss: 0.0100014106
[3,  5000] ,progress 0.9874, mean loss: 0.0089768379

Epoch: 4
[4,  1000] ,progress 0.1973, mean loss: 0.0088870037
[4,  2000] ,progress 0.3948, m

[30,  2000] ,progress 0.3948, mean loss: 0.0014660673
[30,  3000] ,progress 0.5923, mean loss: 0.0019730969
[30,  4000] ,progress 0.7898, mean loss: 0.0015423211
[30,  5000] ,progress 0.9874, mean loss: 0.0010888414

Epoch: 31
[31,  1000] ,progress 0.1973, mean loss: 0.0013168854
[31,  2000] ,progress 0.3948, mean loss: 0.0014319789
[31,  3000] ,progress 0.5923, mean loss: 0.0019390220
[31,  4000] ,progress 0.7898, mean loss: 0.0015099454
[31,  5000] ,progress 0.9874, mean loss: 0.0010574145

Epoch: 32
[32,  1000] ,progress 0.1973, mean loss: 0.0012852806
[32,  2000] ,progress 0.3948, mean loss: 0.0014001582
[32,  3000] ,progress 0.5923, mean loss: 0.0019072065
[32,  4000] ,progress 0.7898, mean loss: 0.0014796975
[32,  5000] ,progress 0.9874, mean loss: 0.0010280525

Epoch: 33


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f9a289458c8>
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/home/al/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f9a289458c8>
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/home/al/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f9a289458c8>
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/s

  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f9a289458c8>
BrokenPipeError: [Errno 32] Broken pipe
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f9a289458c8>
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
  File "/home/al/anaconda3/lib/python3.7/multiproces

BrokenPipeError: [Errno 32] Broken pipe
BrokenPipeError: [Errno 32] Broken pipe
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f9a289458c8>
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
Traceback (most recent call last):
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/home/al/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/al/anaconda3/lib/pytho

[33,  1000] ,progress 0.1973, mean loss: 0.0012557355
[33,  2000] ,progress 0.3948, mean loss: 0.0013703951
[33,  3000] ,progress 0.5923, mean loss: 0.0018774327
[33,  4000] ,progress 0.7898, mean loss: 0.0014513849
[33,  5000] ,progress 0.9874, mean loss: 0.0010005692

Epoch: 34
[34,  1000] ,progress 0.1973, mean loss: 0.0012280479
[34,  2000] ,progress 0.3948, mean loss: 0.0013425087
[34,  3000] ,progress 0.5923, mean loss: 0.0018495170
[34,  4000] ,progress 0.7898, mean loss: 0.0014248425
[34,  5000] ,progress 0.9874, mean loss: 0.0009747839

Epoch: 35
[35,  1000] ,progress 0.1973, mean loss: 0.0012020479
[35,  2000] ,progress 0.3948, mean loss: 0.0013163286
[35,  3000] ,progress 0.5923, mean loss: 0.0018232901
[35,  4000] ,progress 0.7898, mean loss: 0.0013999118
[35,  5000] ,progress 0.9874, mean loss: 0.0009505428

Epoch: 36
[36,  1000] ,progress 0.1973, mean loss: 0.0011775864
[36,  2000] ,progress 0.3948, mean loss: 0.0012916934
[36,  3000] ,progress 0.5923, mean loss: 0.001798

In [17]:
print(torch.tensor(np.nan) == torch.tensor(np.nan))

tensor(0, dtype=torch.uint8)


In [ ]:
print(optimizer.state_dicr())